# Track end
## Imports

In [1]:
import json
from datetime import datetime
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from bson.json_util import dumps
import access

## Functions
### 1.Log function

In [2]:
def log(message):
    """Log function to log errors."""
    timestamp = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S")
    with open("etl.log", "a") as f:
        f.write(timestamp + " " + message + "\n")

### 2.Get tracking records which need to be closed

In [1]:
def containers_at_destination():
    """Find containers which reached point of destination."""
    # Prepare connection
    conn = MongoClient(access.track_end)
    # Query database
    try:
        conn.admin.command("ping")
        cur = conn.one.tracking.aggregate([
            {"$match": {"trackEnd": None}},
            {"$addFields": {"last": {"$last": "$schedule"}}},
            {"$match": {"last.status": "A" }},
            {"$project": {"cntrNo": 1, "_id": 0}}
        ])
        records = json.loads(dumps(cur))
        conn.close()
        if len(records) > 0:
            return records
        else:
            return False
    except ConnectionFailure:
        log("[Tracking closer] [Records to close] "\
            + f"[DB Connection failure]")
        conn.close()
        return False
    except BaseException as err:
        log("[Tracking closer] [Records to close] "\
            + f"[{err.details}]")
        conn.close()
        return False

In [4]:
containers = containers_at_destination()
containers

[{'cntrNo': 'TRIU6671415'}]

### 3.Close tracking records

In [5]:
def set_track_end(data):
    """Set trackEnd field in database to current date and time."""
    if not data:
        return False
    # Prepare connection
    conn = MongoClient(access.track_end)
    # Close records
    try:
        conn.admin.command("ping")
        for item in data:
            cur = conn.one.tracking.update_one(
                {"cntrNo": rec["cntrNo"]},
                {"$set": {"trackEnd": datetime.now().replace(microsecond=0)}},
            )
            if cur.acknowledged == False:
                log("[Tracking closer] [Close] "\
                    + f"[{rec['cntrNo']} not closed in tracking]")
        conn.close()
    except ConnectionFailure:
        log("[Tracking closer] [Records to close] "\
            + f"[DB Connection failure]")
        conn.close()
        return False
    except BaseException as err:
        log("[Tracking closer] [Records to close] "\
            + f"[{err.details}]")
        conn.close()
        return False

In [36]:
set_track_end(containers)

## Pipeline code

In [38]:
records = containers_at_destination()
set_track_end(records)

False

## Debug

In [18]:
# Revert test record back
conn = MongoClient(access.track_end)
cur = conn.one.tracking.update_one(
    {"cntrNo": "TRIU6671415"},
    {"$set": {"trackEnd": None}}
)
conn.close()

In [33]:
# Check that record was closed
records_to_close()

False